In [27]:
import torch
import deep.model as models
import deep.training as training
from tool.preprocessing import DataCollection
from tool.create_dataset import creation

In [30]:
collection = DataCollection(drop_null=True)
dataset,gt = collection.get_dataset(drop_datetime=False) # in dataset is already present the gt value


In [34]:
NUM_RECORD = 10
TOTAL_DIMENSION = NUM_RECORD*6
division = creation(dataset,NUM_RECORD,p=0.75)

mlp_dataset_X_train= division[0][0].flatten(-2)
mlp_dataset_y_train= division[0][1].flatten(-2)[:,0]
mlp_dataset_X_test = division[1][0].flatten(-2)
mlp_dataset_y_test = division[1][1].flatten(-2)[:,0]

In [35]:
import torch.nn as nn
def eval(mlp: nn.Module, data_loader: torch.utils.data.DataLoader, 
             device: torch.device):
  
  loss_fun = nn.MSELoss().to(device)
  correct = 0
  total = 0
  
  with torch.no_grad():
    for x, y in data_loader:
     
      x, y = x.to(device), y.to(device)
      y_pred = mlp(x)
      #print(y_pred.shape,y.shape)
      mean = loss_fun(y_pred,y)
      #print(mean)
      correct+=mean
      
      total+=1
  return correct/total

In [36]:
import torch
import numpy as np
import torch.nn as nn

import torch.optim as optim
import torch.utils.data as data

BATCH_SIZE=512
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Training
model = models.AirMLP(num_fin=60,num_hidden=4096).to(device)
loss_fn = nn.MSELoss(reduction='mean').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)
loader = data.DataLoader(data.TensorDataset(mlp_dataset_X_train, mlp_dataset_y_train), shuffle=True, batch_size=BATCH_SIZE)
loader_test = data.DataLoader(data.TensorDataset(mlp_dataset_X_test, mlp_dataset_y_test), shuffle=False, batch_size=BATCH_SIZE)


n_epochs = 51
for epoch in range(n_epochs):
    
    
    model.train()
    for X_batch, y_batch in loader:
        
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()

        y_pred = model(X_batch)
        print(y_pred.shape)
        loss = loss_fn(y_pred, y_batch)
        #print(y_batch.shape, y_pred.shape)
        loss.backward()

        optimizer.step()
    
    
    
    # Validation
    if epoch % 10 != 0:
        continue
    model.eval()
    print(f"Epoch {epoch} train mse.: {eval(model, loader, device):.3f} test mse.: {eval(model, loader_test, device):.3f}")
    
    

torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])


KeyboardInterrupt: 

In [ ]:
torch.save(model,r'.\results\weights\weights_mlp_20230414.pth')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

ari = collection.ari1952

ari = pd.merge(ari,gt,how="inner", on="valid_at").rename(columns={"pm2p5_y":"pm2p5_t","pm2p5_x":"pm2p5"})

x_test = creation(ari,10)[1][0].flatten(-2)
y_test = creation(ari,10)[1][1].flatten(-2)[:,0]

y_test.isnan().sum()
y_pred = model(x_test[:200,:])[:200]
#y_test=y_test.cpu().detach().numpy()[:200]
y_test=y_test[:200]

for impacts in [y_pred,y_test]:
    timefilteredForce = plt.plot(impacts.cpu().detach().numpy())
    timefilteredForce = plt.xlabel('days')
    timefilteredForce = plt.ylabel('PM2.5')

plt.show()